***

<font color='green'>

# Project Solution: Neural Network - Part 3
### Filename: Signal.csv
### File consists of data with information on various signal tests performed:
- 1. Parameters: Various measurable signal parameters.
- 2. Signal_Quality: Final signal strength or quality

### Context: A communications equipment manufacturing company has a product which is responsible for emitting informative  signals. Company wants to build a machine learning model which can help the company to predict the equipment’s signal quality using various parameters.

### Purpose: Implementing a clickable GUI which can automate Part 1 (NN Regressor) and Part 2 (NN Classifier)

***
***
*Prepared by: Sauvik De*

*Date: March 5, 2021*
    
</font>

***

In [ ]:
# Import Library
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
import re
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, ReLU, LeakyReLU #Flatten, Input
from tensorflow.keras.optimizers import Adam, SGD, Adadelta, RMSprop
from tensorflow.keras.initializers import he_uniform, he_normal, glorot_uniform, glorot_normal, RandomNormal

def r_2_score(y_true, y_pred):
    SS_resid = K.sum(K.square( y_true - y_pred )) 
    SS_total = K.sum(K.square( y_true - K.mean(y_true) ))
    return ( 1 - SS_resid/(SS_total + 0.0000001) )  # 0.0000001 to avoid division by 0

# App window
win = tk.Tk()
win.title('Neural Networks GUI - Great Learning')  # Window Title


# Step 1: Import data frame name ---------------------------------

Name = ttk.Label(win, text="Step 1: File Name")
Name.grid(row=0, column=0, sticky=tk.W)

Name_var = tk.StringVar()
Name_entrybox = ttk.Entry(win, width=16, textvariable=Name_var)
Name_entrybox.grid(row=0, column=1)

def Import_Data():
    global DB
    DF_Name = Name_var.get()
    if((DF_Name != "") and (Path(DF_Name).exists())):
        DB_extension = re.findall("\..*", DF_Name) 
        if DB_extension == ['.xlsx']:
            DB = pd.read_excel(DF_Name)
        elif DB_extension == ['.csv']:
            DB = pd.read_csv(DF_Name)
        # Blank empty window to print confirmation
        confirm = "Done"
    
    else:
        confirm = "File unavailable"
    
    Confirm_entrybox = ttk.Entry(win, width=16)
    Confirm_entrybox.grid(row=0, column=3)
    Confirm_entrybox.insert(1, str(confirm))
    Confirm_entrybox.config(state='readonly')

Import_Data_Button = ttk.Button(win, text="Import Data", command=Import_Data)
Import_Data_Button.grid(row=0, column=2)


# Step 2: Target column name ---------------------------------

Target = ttk.Label(win, text="Step 2: Target Column")
Target.grid(row=1, column=0, sticky=tk.W)

Target_var = tk.StringVar()
Target_entrybox = ttk.Entry(win, width=16, textvariable=Target_var)
Target_entrybox.grid(row=1, column=1)

def Target_Data():
    global DB, X, y, Target_Name, X_train, X_test, y_train, y_test
    Target_Name = Target_var.get()
    
    if('DB' in globals()):
        if(Target_Name != ""):
            Column_name = DB.columns

            for i in range(len(Column_name)):
                if Column_name[i]==Target_Name:
                    confirm="Found"
                    y = DB[Target_Name]                                # Target variable
                    X = DB.drop(Target_Name, axis=1, inplace=False)    # Predictor variable
                    # Train and test split
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, shuffle=True)

                else:
                    confirm="Not Found"
        else:
            confirm="No input target"
    else:
        confirm="No Input File"
                
    Confirm_entrybox = ttk.Entry(win, width=16)
    Confirm_entrybox.grid(row=1, column=3)
    Confirm_entrybox.insert(1, str(confirm))
    Confirm_entrybox.config(state='readonly')

Target_Button = ttk.Button(win, text="Import Target", command=Target_Data)
Target_Button.grid(row=1, column=2)


# Step 3: Modelling ---------------------------------

# ================================================
# 1. Regression
ModellingR = ttk.Label(win, text="Step 3: Neural Network Regressor")
ModellingR.grid(row=2, column=0, sticky=tk.W)

NNR = ttk.Label(win, text="Regression")
NNR.grid(row=3, column=0, sticky=tk.E)

def NN_Regression_Train():
    global regmdl
    if(('X_train' in globals()) & ('X_test' in globals()) & ('y_train' in globals()) & ('y_test' in globals())):
        # Data Pre-processing ...
        # Normalize data
        sc = StandardScaler()
        X_train_sc = sc.fit_transform(X_train)
        X_test_sc  = sc.transform(X_test)
        # Auto-detect number of principal components ...
        cov_matrix = np.cov(X_train_sc.T)
        # Compute Eigen values and Eigen vectors of the covariance matrix
        eig_vals, eig_vecs = np.linalg.eig(cov_matrix)
        # calculate cumulative proportion of variances of eigen vectors
        var_exp = [ (i/sum(eig_vals)) * 100 for i in sorted(eig_vals, reverse=True)]
        cum_var_exp = np.cumsum(var_exp)
        PCcomp = (np.round(cum_var_exp,0) <= 95).sum()
        # using Principal components ...
        
        pca = PCA(n_components=PCcomp)
        X_train_proj = pca.fit_transform(X_train_sc)
        X_test_proj  = pca.transform(X_test_sc)
        
        # Step 1: Design Neural Network regressor
        # Initialize model
        regmdl = Sequential()
        # Input Layer
        regmdl.add(tf.keras.Input(shape=(X_train_proj.shape[1])))
        # Add Hidden Layers with Leaky ReLU activation; Dropout for regularization (to prevent overefitting)
        regmdl.add(Dense(256, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL1'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(256, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL2'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(128, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL3'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(128, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL4'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(64, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL5'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(64, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL6'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(32, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL7'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(32, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL8'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(16, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL9'))
        regmdl.add(Dropout(0.10))
        regmdl.add(Dense(16, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL10'))
        regmdl.add(Dropout(0.10))
        # Output Layer with 1 neuron and linear activation function
        regmdl.add(Dense(1, activation='linear', kernel_initializer=glorot_uniform(seed=1), name='Output'))
        
        # Step 2: Compile model
        regmdl.compile(optimizer=Adam(learning_rate=0.009), loss='mean_squared_error', metrics=[r_2_score])

        # Step 3: Model Fitting
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, mode='min')
        hist = regmdl.fit(X_train_proj, y_train, batch_size=64, epochs=1000, verbose=0,
                          validation_data=(X_test_proj, y_test), callbacks=[callback])
        
        confirm = 'Network Trained'
    
    else:
        confirm = 'Model input unavailable'

    Confirm_entrybox = ttk.Entry(win, width=21)
    Confirm_entrybox.grid(row=3, column=2)
    Confirm_entrybox.insert(1, str(confirm))
    Confirm_entrybox.config(state='readonly')
    
Reg_Train_Button = ttk.Button(win, text="Train", command=NN_Regression_Train)
Reg_Train_Button.grid(row=3, column=1)

# -------------------------------------
NNRP = ttk.Label(win, text="Pickle")
NNRP.grid(row=4, column=0, sticky=tk.E)

def NN_Regression_Pickle():
    if 'regmdl' in globals():
        # serialize model to JSON
        JSONmodel = regmdl.to_json()
        #save the model architecture to JSON file
        with open('nn_regression_model.json', 'w') as JSONfile:
            JSONfile.write(JSONmodel)
        #saving the weights of the model
        regmdl.save_weights('nn_regression_weights.h5')
        
        confirm = 'Saved Model to disk'
        
    else:
        confirm = 'No model available'
    
    Confirm_entrybox = ttk.Entry(win, width=18)
    Confirm_entrybox.grid(row=4, column=2)
    Confirm_entrybox.insert(1, str(confirm))
    Confirm_entrybox.config(state='readonly')

Reg_Pkl_Button = ttk.Button(win, text="Run", command=NN_Regression_Pickle)
Reg_Pkl_Button.grid(row=4, column=1)

# ================================================
# 2. Classification

ModellingC = ttk.Label(win, text="Step 4: Neural Network Classifier")
ModellingC.grid(row=5, column=0, sticky=tk.W)

NNC = ttk.Label(win, text="Classifier")
NNC.grid(row=6, column=0, sticky=tk.E)

def NN_Classification_Train():
    global clfmdl
    if(('X_train' in globals()) & ('X_test' in globals()) & ('y_train' in globals()) & ('y_test' in globals())):
        # Data Pre-processing ...
        # Normalize data
        sc = StandardScaler()
        X_train_sc = sc.fit_transform(X_train)
        X_test_sc  = sc.transform(X_test)
        # Auto-detect number of principal components ...
        cov_matrix = np.cov(X_train_sc.T)
        # Compute Eigen values and Eigen vectors of the covariance matrix
        eig_vals, eig_vecs = np.linalg.eig(cov_matrix)
        # calculate cumulative proportion of variances of eigen vectors
        var_exp = [ (i/sum(eig_vals)) * 100 for i in sorted(eig_vals, reverse=True)]
        cum_var_exp = np.cumsum(var_exp)
        PCcomp = (np.round(cum_var_exp,0) <= 95).sum()
        # using Principal components ...
        pca = PCA(n_components=PCcomp)
        X_train_proj = pca.fit_transform(X_train_sc)
        X_test_proj  = pca.transform(X_test_sc)
        # to apply One-Hot-Encoding, we first shift class labels to start from 0
        y_train_sh = y_train - y_train.min()
        y_test_sh  = y_test  - y_test.min()
        # One-Hot-Encode target column for signal strength 
        y_train_enc = tf.keras.utils.to_categorical(y_train_sh, num_classes=y_train.nunique())
        y_test_enc  = tf.keras.utils.to_categorical(y_test_sh, num_classes=y_test.nunique())
        
        # Step 1: Design Neural Network classifier
        # Initialize model
        clfmdl = Sequential()
        # Input Layer
        clfmdl.add(tf.keras.Input(shape=(X_train_proj.shape[1])))
        # Add Hidden Layers with Leaky ReLU activation; Dropout for regularization (to prevent overefitting)
        clfmdl.add(Dense(256, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL1'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(256, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL2'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(128, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL3'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(128, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL4'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(64, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL5'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(64, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL6'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(32, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL7'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(32, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL8'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(16, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL9'))
        clfmdl.add(Dropout(0.10))
        clfmdl.add(Dense(16, activation=LeakyReLU(), kernel_initializer=glorot_uniform(seed=1), name='HL10'))
        clfmdl.add(Dropout(0.10))
        # Output Layer with 6 neurons (equal to number of target classes) and softmax activation function
        clfmdl.add(Dense(y_train.nunique(), activation='softmax', kernel_initializer=glorot_uniform(seed=1), name='Output'))
        
        # Step 2: Compile model
        clfmdl.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

        # Step 3: Model Fitting
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, mode='min')
        hist = clfmdl.fit(X_train_proj, y_train_enc, batch_size=64, epochs=1000, verbose=0,
                          validation_data=(X_test_proj, y_test_enc), callbacks=[callback])
        
        confirm = 'Network Trained'
    
    else:
        confirm = 'Model input unavailable'

    Confirm_entrybox = ttk.Entry(win, width=21)
    Confirm_entrybox.grid(row=6, column=2)
    Confirm_entrybox.insert(1, str(confirm))
    Confirm_entrybox.config(state='readonly')

Clf_Train_Button = ttk.Button(win, text="Train", command=NN_Classification_Train)
Clf_Train_Button.grid(row=6, column=1)

# -------------------------------------
NNCP = ttk.Label(win, text="Pickle")
NNCP.grid(row=7, column=0, sticky=tk.E)

def NN_Classification_Pickle():
    if 'clfmdl' in globals():
        # serialize model to JSON
        JSONmodel = clfmdl.to_json()
        #save the model architecture to JSON file
        with open('nn_classification_model.json', 'w') as JSONfile:
            JSONfile.write(JSONmodel)
        #saving the weights of the model
        clfmdl.save_weights('nn_classification_weights.h5')
        
        confirm = 'Saved Model to disk'
        
    else:
        confirm = 'No model available'
    
    Confirm_entrybox = ttk.Entry(win, width=18)
    Confirm_entrybox.grid(row=7, column=2)
    Confirm_entrybox.insert(1, str(confirm))
    Confirm_entrybox.config(state='readonly')

Clf_Pkl_Button = ttk.Button(win, text="Run", command=NN_Classification_Pickle)
Clf_Pkl_Button.grid(row=7, column=1)

win.mainloop()

Using TensorFlow backend.
